# LangChain + GitHub Models + Azure OpenAI

LangChain is a modular framework designed to streamline the development and integration of applications utilizing large language models (LLMs) by providing tools for effective prompt engineering, data handling, and complex workflows.

LangChain supports a number of model providers including Azure OpenAI and Cohere. Below find examples of how LangChain can be used with these models provided by the GitHub Models service.


## 1. Install dependencies

In [1]:
%pip install langchain-openai
%pip install langchain-community
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.37.2
    Uninstalling openai-1.37.2:
      Successfully uninstalled openai-1.37.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.7 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 59.4 MB/s et

## 2. Constructing LangChain's ChatOpenAI class

Setting up the OpenAI API key and base URL for LangChain is the same as for OpenAI's Python client. You can set the `OPENAI_API_KEY` and `OPENAI_API_BASE` environment variables, or pass them directly to the `ChatOpenAI` class.


In [2]:
from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

if not os.getenv("GITHUB_TOKEN"):
    raise ValueError("GITHUB_TOKEN is not set")

os.environ["OPENAI_API_KEY"] = os.getenv("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://models.inference.ai.azure.com/"

GPT_MODEL = "gpt-4o-mini"

llm = ChatOpenAI(model=GPT_MODEL)

Now you can use the ChatOpenAI class to interact with the OpenAI API. 

In [3]:
from IPython.display import Markdown, display

response = llm.invoke("What is the meaning of life?")
display(Markdown(response.content))

The meaning of life is a deeply philosophical question that has been explored by thinkers, theologians, and writers throughout history. Answers can vary widely depending on cultural, religious, and personal beliefs. Here are a few perspectives:

1. **Philosophical**: Many philosophers suggest that the meaning of life is about the pursuit of happiness, knowledge, and self-fulfillment. Existentialists, for instance, argue that individuals create their own meaning through choices and actions.

2. **Religious**: Various religions offer distinct interpretations. For example, in Christianity, the meaning of life may revolve around serving God and achieving eternal life. In Buddhism, it might be about attaining enlightenment and overcoming suffering.

3. **Scientific**: From a biological perspective, some might argue that the purpose of life is to survive and reproduce, ensuring the continuation of one’s genes.

4. **Personal**: Many people find meaning through relationships, experiences, and contributions to society. This could involve pursuing passions, helping others, or creating art.

Ultimately, the meaning of life is subjective, and many find that it evolves over time based on experiences and reflections. What resonates with one person may differ greatly from another.

## 3. Use a prompt template

LangChain has the concept of a prompt template that allows you parameterize the prompt and fill in the parameters with values. 

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a fun assistant that uses a lot of emojis."),
    ("user", "{input}")
])

Next, we combine the promt with the ChatOpenAI into a chain:

In [5]:
chain = prompt | llm 

Then we invoke the chain with the input data:

In [6]:
chain.invoke({"input": "when was the first ESC held?"})


AIMessage(content='The first Eurovision Song Contest (ESC) was held on May 24, 1956! 🎤🎶 It took place in Lugano, Switzerland, and featured seven countries. Each country presented two songs! 🌍✨ What a fun way to start an amazing tradition! 🎉', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 30, 'total_tokens': 88, 'completion_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_80a1bad4c7', 'finish_reason': 'stop', 'logprobs': None}, id='run-344db5f2-af9f-4fbd-aae4-fcc85c82e79f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 58, 'total_tokens': 88})

## 4. Add a parsing step to your chain

We can also extend the chain to include additional steps, such as parsing the content from the reponse:

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "when was the first ESC held?"})

'The first Eurovision Song Contest (ESC) was held on May 24, 1956! 🎤🌍 It took place in Lugano, Switzerland, and featured just seven countries competing. What a fun way to kick off a legendary tradition! 🎶✨'

## 5. Use retrieval to ground the model

Now let's ask for some more recent information:

In [8]:
chain.invoke({"input": f"Who won the 2024 ESC?"})

"I don't have the latest updates on events after my last knowledge cutoff. 🎤✨ For the most accurate info on the 2024 Eurovision Song Contest, I'd recommend checking the official Eurovision website or social media! 🌍🎶 Got any favorite Eurovision songs? 🎵💖"

Looks like the model is not up to date with the latest contest. Let's help it out with some content from the wikipedia:

In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://en.wikipedia.org/wiki/Eurovision_Song_Contest_2024")

docs = loader.load()
len(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


ModuleNotFoundError: No module named 'bs4'

Due to the limits of the free service, the content retrieved from Wikipedia is too large to be added to the context -- that is ok since we can use a vector store to index the content. For that we need an embedding model which we can use with the OpenAIEmbeddings class. Note that we need to reduce the chunk size due to the token limits of the GitHub Models free service. 

In [ ]:
from langchain_openai import OpenAIEmbeddings

EMBEDDINGS_MODEL = "text-embedding-3-small"

# need to constrain the chunk_size to work within the limits of the GitHub Model service
embeddings = OpenAIEmbeddings(model=EMBEDDINGS_MODEL,chunk_size=30)

Now, we will use the [FAISS](https://github.com/facebookresearch/faiss) library to index the vectors for a similarity search.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Next, we can create a document chain that will use some context and the user's question to provide an answer:

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""You are a fun assistant that uses a lot of emojis.
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We could just provide the context directly to the chain like so:

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Who won 2024 ESC?",
    "context": [Document(page_content="The top 3 placings in the 2024 Eurovision Song Contest were:\n1. Switzerland\n2. Croatia\n3. Ukraine")]
})

But we really want a retrieval chain that will retrieve the right documents from our vector index and then uses it as context in our prompt like so:

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

Let's ask this one who won the 2024 ESC:

In [ ]:
response = retrieval_chain.invoke({"input": f"Who won the 2024 ESC?"})
print(response["answer"])


And a few more increasingly detailed questions:

In [ ]:
response = retrieval_chain.invoke({"input": "where did the 2024 ESC take place?"})
print(response["answer"])

In [ ]:
response = retrieval_chain.invoke({"input": "who designed the stage for the 2024 ESC?"})
print(response["answer"])

In [ ]:
response = retrieval_chain.invoke({"input": "when did the semi-finals for the 2024 ESC take place?"})
print(response["answer"])